[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/Hunyuan3D-2.1-jupyter/blob/main/Hunyuan3D-2.1_jupyter.ipynb)

In [ ]:
%cd /content
!git clone -b dev https://github.com/camenduru/Hunyuan3D-2.1-hf

%cd /content/Hunyuan3D-2.1/hy3dshape
!pip install -r requirements.txt

%cd /content/Hunyuan3D-2.1/hy3dpaint
!pip install bpy realesrgan pytorch-lightning timm fast-simplification
!pip install /content/Hunyuan3D-2.1/custom_rasterizer-0.1-cp310-cp310-linux_x86_64.whl
!pip install /content/Hunyuan3D-2.1/hy3dpaint/packages/custom_rasterizer

%cd /content/Hunyuan3D-2.1/hy3dpaint/DifferentiableRenderer
!bash compile_mesh_painter.sh

!pip uninstall BasicSR -y
!pip install git+https://github.com/XPixelGroup/BasicSR

!sed -i 's/courent = courent\.simplify_quadric_decimation(target_count)/courent = courent.simplify_quadric_decimation(face_count=target_count)/g' /content/Hunyuan3D-2.1/hy3dpaint/utils/simplify_mesh_utils.py

In [ ]:
%cd /content/Hunyuan3D-2.1/hy3dshape

from hy3dshape.pipelines import Hunyuan3DDiTFlowMatchingPipeline
  
shape_pipeline = Hunyuan3DDiTFlowMatchingPipeline.from_pretrained('tencent/Hunyuan3D-2.1')
image_path = "/content/Hunyuan3D-2.1/assets/example_images/1204.png"
mesh_untextured = shape_pipeline(image=image_path)[0]
!mkdir '/content/Hunyuan3D-2.1/test1'
mesh_untextured.export('/content/Hunyuan3D-2.1/test1/test.glb')

%cd /content/Hunyuan3D-2.1/hy3dpaint

from textureGenPipeline import Hunyuan3DPaintPipeline, Hunyuan3DPaintConfig

try:
    from utils.torchvision_fix import apply_fix
    apply_fix()
except ImportError:
    print("Warning: torchvision_fix module not found, proceeding without compatibility fix")
except Exception as e:
    print(f"Warning: Failed to apply torchvision fix: {e}")

paint_pipeline = Hunyuan3DPaintPipeline(Hunyuan3DPaintConfig(max_num_view=6, resolution=512))
mesh_path = "/content/Hunyuan3D-2.1/test1/test.glb"
mesh_textured = paint_pipeline(
    mesh_path=mesh_path,
    image_path=image_path,
    use_remesh=True
)